# Times Table

An example of using `np.outer` to compute a 10x10 grid of pairwise multiplications, each
with a separate product network.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import nengo
from nengo_gui.jupyter import InlineGUI

import gyrus

## Nengo Approach

Here is how you would do this using Nengo.

In [ ]:
u = np.linspace(-1, 1, 10)

In [ ]:
with nengo.Network() as model:
    stims = [nengo.Node(u_i) for u_i in u]
    probes = np.empty((len(u), len(u)), dtype=object)
    for i in range(len(u)):
        for j in range(len(u)):
            product = nengo.networks.Product(n_neurons=200, dimensions=1)
            nengo.Connection(stims[i], product.input_a, synapse=None)
            nengo.Connection(stims[j], product.input_b, synapse=None)
            probes[i, j] = nengo.Probe(product.output, synapse=0.005)

In [ ]:
with nengo.Simulator(model) as sim:
    sim.run(0.1)

out = np.asarray(
    [
        [sim.data[probes[i, j]].squeeze(axis=-1) for j in range(len(u))]
        for i in range(len(u))
    ]
)

In [ ]:
def plot(out):
    # Reshape the data into an n-by-n block matrix where each bock
    # is s-by-s corresponding to the s**2 time-points for the
    # output of the corresponding Product network.
    n = len(u)
    assert n == out.shape[0] == out.shape[1]
    s = int(np.sqrt(out.shape[2]))
    assert s ** 2 == out.shape[2]

    a = np.zeros((n * s, n * s))
    for i in range(n):
        for j in range(n):
            a[i * s : (i + 1) * s, j * s : (j + 1) * s] = out[i, j].reshape((s, s))

    u_lim = (np.min(u), np.max(u))
    fig, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 6))
    axes[0].set_title("Model Output (Reshaped)")
    axes[0].imshow(a, extent=u_lim * 2)
    axes[1].set_title("Ideal Times Table")
    axes[1].imshow(np.outer(u, u), extent=u_lim * 2)
    fig.show()

In [ ]:
plot(out)

## Gyrus Approach

And, here is the exact same Nengo network generated using Gyrus. It is essentially just
one line: `np.outer(x, x).filter(tau)`!

In [ ]:
def times_table(u, tau=0.005):
    x = gyrus.stimuli(u)
    return np.outer(x, x).filter(tau)


out = np.asarray(times_table(u).run(0.1)).squeeze(axis=-1)

In [ ]:
plot(out)

## Nengo GUI Visualization

Lastly, we visualize a 3x3 slice of the model in the Nengo GUI. Note that the only the
portion of the operator graph that is sliced out is generated into the
`nengo.Network()`.

In [ ]:
with nengo.Network() as model:
    times_table(u)[:3, :3].make()

InlineGUI(model)